In [1]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
  return (year - 1996) * 12 + month - 6

n_features = 12 # or 54
lookback = 3
chosen_stocks = ["PG"] # "AMZN", "MSFT", "IBM", "INTC", "QCOM", "NVDA", "IBM", "ADBE", "EBAY", "CSCO"

In [2]:
dfFeature = pd.read_csv("./lstm_2004_12.csv")
# dfFeature.loc[dfFeature["return"] == 0, "return"] = 1
dfFeature = dfFeature[dfFeature.QAId.isin(chosen_stocks)]
features = dfFeature.drop(columns=['month_id', 'QAId']).as_matrix()
print(features.shape)

(157, 12)


In [3]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features) \
                                   .reshape(157, len(chosen_stocks), n_features) \
                                   .reshape(157, len(chosen_stocks) * n_features)
print(normalizedFeatures.shape)

(157, 12)


In [4]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
# dfTarget.loc[dfTarget["return"] == 0, "return"] = 1
dfTarget = dfTarget[dfTarget.QAId.isin(chosen_stocks)]
dfTarget = dfTarget[dfTarget.month_id >= (to_month_id(2004, 1) + lookback)]
targets = MinMaxScaler().fit_transform(dfTarget.drop(columns=['month_id', 'QAId']).as_matrix())
y = targets.reshape(157 - lookback + 1, len(chosen_stocks))
print(y.shape)

(155, 1)


In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = normalizedFeatures
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(157, 12)


In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator

x = np.zeros((157 - lookback + 1, lookback, n_features * len(chosen_stocks)))
y_mock = np.zeros((157, len(chosen_stocks)))

i = 0
for train, test in TimeseriesGenerator(splittedFeature, y_mock, length=lookback, batch_size=1):
  if i > 157 - lookback:
    break
  x[i] = train[0]
  i = i + 1

In [7]:
from sklearn.model_selection import PredefinedSplit

test_months = 26
end_point = 157 - lookback + 1
split_point = 157 - lookback + 1 - test_months

x_train = x[0:split_point - 48].reshape(split_point - 48, lookback, n_features * len(chosen_stocks))
y_train = y[0:split_point - 48].reshape(split_point - 48, len(chosen_stocks))
x_validate = x[split_point - 36:split_point].reshape(36, lookback, n_features * len(chosen_stocks))
y_validate = y[split_point - 36:split_point].reshape(36, len(chosen_stocks))
x_test = x[split_point:end_point].reshape(test_months, lookback, n_features * len(chosen_stocks))
y_test = y[split_point:end_point].reshape(test_months, len(chosen_stocks))

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

# ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

# for train_ids, test_ids in ps:
#   print(str(train_ids) + ", " + str(test_ids))

(81, 3, 12)
(81, 1)
(36, 3, 12)
(36, 1)
(26, 3, 12)
(26, 1)


In [10]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, CuDNNLSTM
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(lookback, n_features * len(chosen_stocks)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(len(chosen_stocks), activation="relu"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=0.0), metrics=['accuracy'], *param)
    return model

In [11]:
batch_size=30

model = create_model()
train_score = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), batch_size=batch_size, epochs=200)
print(train_score)

Train on 81 samples, validate on 36 samples
Epoch 1/200
81/81 [==============================] - 2s 27ms/step - loss: 4.4671 - acc: 0.4815 - val_loss: 3.1357 - val_acc: 0.3889
Epoch 2/200
81/81 [==============================] - 0s 565us/step - loss: 2.5632 - acc: 0.4815 - val_loss: 2.7037 - val_acc: 0.3889
Epoch 3/200
81/81 [==============================] - 0s 572us/step - loss: 2.2413 - acc: 0.4815 - val_loss: 2.4967 - val_acc: 0.3889
Epoch 4/200
81/81 [==============================] - 0s 571us/step - loss: 2.1193 - acc: 0.4815 - val_loss: 2.3745 - val_acc: 0.3889
Epoch 5/200
81/81 [==============================] - 0s 552us/step - loss: 2.0119 - acc: 0.4815 - val_loss: 2.2852 - val_acc: 0.3889
Epoch 6/200
81/81 [==============================] - 0s 547us/step - loss: 1.9347 - acc: 0.4815 - val_loss: 2.2118 - val_acc: 0.3889
Epoch 7/200
81/81 [==============================] - 0s 540us/step - loss: 1.8649 - acc: 0.4815 - val_loss: 2.1484 - val_acc: 0.3889
Epoch 8/200
81/81 [=======

Epoch 62/200
81/81 [==============================] - 0s 579us/step - loss: 0.7212 - acc: 0.4815 - val_loss: 0.7764 - val_acc: 0.3889
Epoch 63/200
81/81 [==============================] - 0s 553us/step - loss: 0.7210 - acc: 0.4815 - val_loss: 0.7667 - val_acc: 0.3889
Epoch 64/200
81/81 [==============================] - 0s 558us/step - loss: 0.7046 - acc: 0.5062 - val_loss: 0.7580 - val_acc: 0.3889
Epoch 65/200
81/81 [==============================] - 0s 552us/step - loss: 0.7146 - acc: 0.5062 - val_loss: 0.7504 - val_acc: 0.3889
Epoch 66/200
81/81 [==============================] - 0s 532us/step - loss: 0.7136 - acc: 0.4815 - val_loss: 0.7438 - val_acc: 0.3889
Epoch 67/200
81/81 [==============================] - 0s 535us/step - loss: 0.7131 - acc: 0.4691 - val_loss: 0.7382 - val_acc: 0.3889
Epoch 68/200
81/81 [==============================] - 0s 538us/step - loss: 0.6959 - acc: 0.5309 - val_loss: 0.7332 - val_acc: 0.3889
Epoch 69/200
81/81 [==============================] - 0s 544us

Epoch 123/200
81/81 [==============================] - 0s 594us/step - loss: 0.6856 - acc: 0.5679 - val_loss: 0.7089 - val_acc: 0.3333
Epoch 124/200
81/81 [==============================] - 0s 545us/step - loss: 0.6901 - acc: 0.5062 - val_loss: 0.7088 - val_acc: 0.3333
Epoch 125/200
81/81 [==============================] - 0s 534us/step - loss: 0.6903 - acc: 0.5185 - val_loss: 0.7090 - val_acc: 0.3333
Epoch 126/200
81/81 [==============================] - 0s 508us/step - loss: 0.6842 - acc: 0.5802 - val_loss: 0.7088 - val_acc: 0.3333
Epoch 127/200
81/81 [==============================] - 0s 718us/step - loss: 0.6854 - acc: 0.5432 - val_loss: 0.7091 - val_acc: 0.3333
Epoch 128/200
81/81 [==============================] - 0s 745us/step - loss: 0.6861 - acc: 0.5309 - val_loss: 0.7089 - val_acc: 0.3333
Epoch 129/200
81/81 [==============================] - 0s 533us/step - loss: 0.6772 - acc: 0.5556 - val_loss: 0.7089 - val_acc: 0.3333
Epoch 130/200
81/81 [==============================] - 

Epoch 184/200
81/81 [==============================] - 0s 552us/step - loss: 0.6782 - acc: 0.6296 - val_loss: 0.7133 - val_acc: 0.3611
Epoch 185/200
81/81 [==============================] - 0s 564us/step - loss: 0.6815 - acc: 0.5802 - val_loss: 0.7139 - val_acc: 0.3611
Epoch 186/200
81/81 [==============================] - 0s 564us/step - loss: 0.6859 - acc: 0.5556 - val_loss: 0.7139 - val_acc: 0.3611
Epoch 187/200
81/81 [==============================] - 0s 549us/step - loss: 0.6767 - acc: 0.6296 - val_loss: 0.7151 - val_acc: 0.3611
Epoch 188/200
81/81 [==============================] - 0s 551us/step - loss: 0.6777 - acc: 0.5926 - val_loss: 0.7150 - val_acc: 0.3611
Epoch 189/200
81/81 [==============================] - 0s 575us/step - loss: 0.6845 - acc: 0.5926 - val_loss: 0.7148 - val_acc: 0.3611
Epoch 190/200
81/81 [==============================] - 0s 591us/step - loss: 0.6808 - acc: 0.5679 - val_loss: 0.7146 - val_acc: 0.3611
Epoch 191/200
81/81 [==============================] - 

In [12]:
import matplotlib.pyplot as plt

test_score = model.evaluate(x_test, y_test, batch_size=batch_size)
for i, mn in enumerate(model.metrics_names):
  print(mn + ": " + str(test_score[i]))

plt.plot(train_score.history["loss"])
plt.plot(train_score.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Metrics")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

26/26 [==============================] - 0s 149us/step
loss: 0.6811535358428955
acc: 0.5769230723381042


In [13]:
from sklearn.metrics import precision_score, recall_score, classification_report

def threshold_tester(result, target, a, b):
  for t in np.arange(a, b + 0.05, 0.05):
    print("thresholder: " + str(t))
    print(classification_report(target, (result >= t).astype(int)))

In [14]:
result = model.predict(x_test, batch_size=batch_size)
print(result)
threshold_tester(result, y_test, 0.1, 0.9)

[[0.5116248 ]
 [0.47382477]
 [0.43113175]
 [0.4219654 ]
 [0.4225787 ]
 [0.43635398]
 [0.4214617 ]
 [0.41693527]
 [0.39348117]
 [0.39805683]
 [0.46968994]
 [0.51771545]
 [0.5136954 ]
 [0.5051208 ]
 [0.50953877]
 [0.49568444]
 [0.46939042]
 [0.48248374]
 [0.5048509 ]
 [0.5322045 ]
 [0.5238401 ]
 [0.5499815 ]
 [0.5265681 ]
 [0.54254735]
 [0.5129019 ]
 [0.5543052 ]]
thresholder: 0.1
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0       0.54      1.00      0.70        14

avg / total       0.29      0.54      0.38        26

thresholder: 0.15000000000000002
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0       0.54      1.00      0.70        14

avg / total       0.29      0.54      0.38        26

thresholder: 0.20000000000000004
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0    

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
import math

buy_list = []
sell_list = []

for j, stock in enumerate(chosen_stocks):
  print(stock)
  sorted_result = sorted(map(lambda x: x[j], result))
  midpt = (sorted_result[-2] + sorted_result[1]) / 2
  upper_threshold = midpt * 1.05
  lower_threshold = midpt * 0.95
  
  print("Target\tPredict\tConsequence")
  for i, r in enumerate(result):
    prediction = r[j].item()
    target = y_test[i][j].item()
    buy_or_sell = 1 if prediction > upper_threshold else (-1 if prediction < lower_threshold else 0)
    if prediction > upper_threshold:
      buy_list.append({'month_id': i + 223, 'QAId': stock})
    if prediction < lower_threshold:
      sell_list.append({'month_id': i + 223, 'QAId': stock})
    
    to_print = str(target) + "\t" + str(buy_or_sell)
    if (buy_or_sell == -1 and target == 0) or (buy_or_sell == 1 and target == 1):
      print(to_print + "\tGain")
    elif (buy_or_sell == -1 and target == 1) or (buy_or_sell == 1 and target == 0):
      print(to_print + "\tLoss")
    else:
      print(to_print + "\tNothing")
  print("\n")

print(buy_list)
print(sell_list)

PG
Target	Predict	Consequence
1.0	1	Gain
0.0	0	Nothing
0.0	-1	Gain
0.0	-1	Gain
0.0	-1	Gain
0.0	-1	Gain
0.0	-1	Gain
1.0	-1	Loss
1.0	-1	Loss
0.0	-1	Gain
1.0	0	Nothing
1.0	1	Gain
0.0	1	Loss
1.0	1	Gain
0.0	1	Loss
1.0	0	Nothing
1.0	0	Nothing
1.0	0	Nothing
1.0	1	Gain
1.0	1	Gain
0.0	1	Loss
0.0	1	Loss
1.0	1	Gain
1.0	1	Gain
1.0	1	Gain
0.0	1	Loss


[{'month_id': 223, 'QAId': 'PG'}, {'month_id': 234, 'QAId': 'PG'}, {'month_id': 235, 'QAId': 'PG'}, {'month_id': 236, 'QAId': 'PG'}, {'month_id': 237, 'QAId': 'PG'}, {'month_id': 241, 'QAId': 'PG'}, {'month_id': 242, 'QAId': 'PG'}, {'month_id': 243, 'QAId': 'PG'}, {'month_id': 244, 'QAId': 'PG'}, {'month_id': 245, 'QAId': 'PG'}, {'month_id': 246, 'QAId': 'PG'}, {'month_id': 247, 'QAId': 'PG'}, {'month_id': 248, 'QAId': 'PG'}]
[{'month_id': 225, 'QAId': 'PG'}, {'month_id': 226, 'QAId': 'PG'}, {'month_id': 227, 'QAId': 'PG'}, {'month_id': 228, 'QAId': 'PG'}, {'month_id': 229, 'QAId': 'PG'}, {'month_id': 230, 'QAId': 'PG'}, {'month_id': 231, 'QAId': 'PG'

In [16]:
dfBuy = pd.DataFrame(buy_list, columns=["month_id", "QAId"])
dfSell = pd.DataFrame(sell_list, columns=["month_id", "QAId"])
dfBuy.head(5)

,month_id,QAId
0,223,PG
1,234,PG
2,235,PG
3,236,PG
4,237,PG


In [17]:
dfSell.head(5)

,month_id,QAId
0,225,PG
1,226,PG
2,227,PG
3,228,PG
4,229,PG


In [18]:
filename_base = "_".join([stock.lower() for stock in chosen_stocks])
# filename_model = "./" + filename_base + "_model.h5"
# filename_weights = "./" + filename_base + "_weights.h5"
filename_output_buy = "./" + filename_base + "_output_buy.h5"
filename_output_sell = "./" + filename_base + "_output_sell.h5"

# model.save(filename_model)
# model.save_weights(filename_weights)
dfBuy.to_csv(filename_output_buy, index=False)
dfSell.to_csv(filename_output_sell, index=False)

# files.download(filename_model)
# files.download(filename_weights)
# files.download(filename_output_buy)
# files.download(filename_output_sell)